# LLMのRAGを使ったQABOT

1. **データ取得**:
    - ICML 2024の論文PDFを取得し、./icml_2024_papersディレクトリに格納する
2. **データの埋め込みベクトル化**:
    - 取得したPDFからテキストを抽出し、ベクトル化する。
3. **LLMのRAGモデルの設定**:    <---- now
    - 質問に対して関連する文書を検索し、その文書を基に回答を生成するRAGモデルを設定する。
    1. 関連する文章自体の概要を出力する
    
4. **QABOTの構築**:
    - ユーザーインターフェースを作成し、ユーザーからの質問に対してRAGモデルを使って回答するQABOTを構築する。

In [1]:
!pip3 install python-dotenv tiktoken
!pip3 install openai==1.3.4
!pip3 install scipy

In [2]:
!pip3 install langdetect==1.0.9
!pip3 install lingua-language-detector==2.0.2
!pip3 install PyPDF2

## 質問に対して関連する文書を検索し、その文書を基に回答を生成する

In [3]:
import os
import pandas as pd
import ast
from scipy import spatial
from tqdm import tqdm
from PyPDF2 import PdfReader
from langdetect import detect

import openai
from openai import OpenAI
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
openai.api_key = os.getenv("OPENAI_API_KEY")

MODEL_NAME = "gpt-3.5-turbo-0125"
# MODEL_NAME = "gpt-3.5-turbo-instruct"
# MODEL_NAME = "gpt-4-0125-preview"
# MODEL_NAME = "gpt-4-turbo-2024-04-09"
MODEL4o_NAME = "gpt-4o-2024-05-13"

# モデルとエンコーディングの設定
# EMBEDDING_MODEL = "text-embedding-ada-002"
EMBEDDING_MODEL = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
# max_tokens = 8000

max_tokens = 1000
TEMPERATURE = 0.7

# OpenAIクライアントの初期化
client = OpenAI()

In [6]:
# 関連度を計算するための関数
def relatedness_fn(x, y):
    return 1 - spatial.distance.cosine(x, y)

In [7]:
# search function
def strings_ranked_by_relatedness(query: str, csv_files, top_n: int = 3):
    related_texts = []
    query_embedding_response = client.embeddings.create(model=EMBEDDING_MODEL, input=query)
    query_embedding = query_embedding_response.data[0].embedding
    for csv_file in csv_files:
        df = pd.read_csv(csv_file, converters={'embedding': ast.literal_eval})

        strings_and_relatednesses = [
            (row["text"], relatedness_fn(query_embedding, row["embedding"]), csv_file)
            for _, row in df.iterrows()
        ]
        related_texts.extend(strings_and_relatednesses)

    related_texts.sort(key=lambda x: x[1], reverse=True)
    # print(related_texts[:2])
    strings, relatednesses, files = zip(*related_texts[:top_n])
    return strings, relatednesses, files

In [8]:
# GPT-3.5で回答を生成
def generate_response(query: str, csv_files, top_n=3):
    strings, relatednesses, files = strings_ranked_by_relatedness(query, csv_files, top_n=top_n)
    context = "\n\n".join([f"File: {file}\nText: {text}" for text, file in zip(strings, files)])
    prompt = [
        {'role': 'system', 'content': 'You are an expert assistant who answers questions based on provided documents.'},
        {'role': 'user', 'content': f"Answer the following question based on the provided context.\n\nQuestion: {query}\n\nContext:\n{context}"}
    ]
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=prompt,
        temperature=0.7,
    )
    # 重複ファイル名削除チェック
    used_files = set()
    for file, text in zip(files, strings):
        if file not in used_files:
            # print(f"File: {file}")
            used_files.add(file)
    return response.choices[0].message.content, used_files, strings

In [9]:
save_dir = "./icml_2024_embeddings"
csv_dir = save_dir
# csv_files = os.listdir(save_dir)
csv_files = [os.path.join(csv_dir, file) for file in os.listdir(csv_dir) if file.endswith('.csv')]
# csv_files

In [10]:
%%time
# 質問に対する回答を生成
query = "RAGを使ったフレームワークやLLMオーケストレーションフレームワーク、Agentに関連する提案内容についてその技術的な内容を順番に過不足なく説明してください"
response, files, strings = generate_response(query, csv_files, top_n=3)

CPU times: user 30.5 s, sys: 289 ms, total: 30.8 s
Wall time: 37.9 s


In [11]:
# linguaを使用するとmarkdown記法を英語と判断してしまうのでナイーブベースで言語取得
lang = detect(response)
print(f"language: {lang}")

language: en


In [12]:
def translate_to_japanese(model_name, detailed_outline):
    """
    Translates the detailed outline of a Wikipedia page from English to Japanese.
    
    Parameters:
    model_name (str): The model to be used for translation.
    detailed_outline (str): The detailed outline in English.
    
    Returns:
    str: The translated outline in Japanese.
    """
    prompt = [
        {"role": "system", "content": "You need to translate the following English text into Japanese."},
        {"role": "user", "content": detailed_outline},
        {"role": "system", "content": "Please provide the translation of the entire text into Japanese, maintaining the accuracy and context of the original information."}
    ]
    
    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        temperature=TEMPERATURE,
    )
    
    translated_outline = response.choices[0].message.content
    
    return translated_outline

In [13]:
%%time
if lang != "ja":
    translated_response = translate_to_japanese(MODEL_NAME, response)
else:
    translated_response = response

CPU times: user 8.67 ms, sys: 0 ns, total: 8.67 ms
Wall time: 8.94 s


In [14]:
# 回答と共に使用したファイル名とテキストを表示
print(f"Response: {translated_response}")
print("\nSources used:")
for file, text in zip(files, strings):
    print(f"File: {file}")
    # print(f"Text: {text}\n")

Response: RAGフレームワーク、LLMオーケストレーションフレームワーク、およびエージェントに関連する提案に関連する技術内容を包括的に説明するためには、以下の文書で提示された順序に従うことができます：

1. **RAGフレームワーク**:
   - RAG（Retrieval-Augmented Generation）戦略は、現在の会話コンテキストに整合するダイアログ例を選択することを含みます。応答を生成するために追加の知識ベースを参照します。
   - RAGのバリエーションには、構造化データソースなどの複雑なコンポーネントを組み込むために開発されたものがあります。たとえば、RET-LLMはパーソナライズされた知識グラフメモリを構築し、SUGREは知識グラフから関連するサブグラフを埋め込むためにGraph Neural Networksを使用します。
   - Knowl-edgeGPTは、コード形式の知識ベースの検索クエリを生成し、事前定義されたKB操作関数を含みます。
   - 提案されたDFA-RAGフレームワークは、トレーニングダイアログから学習されたDefinite Finite Automaton（DFA）をLLM内に統合し、会話エージェントの能力を向上させます。
   - DFA-RAGは、人間が読み取れるDFAを通じて解釈可能な構造、応答のためのコンテキストに応じた検索、および既存のLLMとのプラグアンドプレイ互換性を提供します。

2. **LLMオーケストレーションフレームワーク**:
   - ドメイン固有のダイアログ生成に焦点を当てたタスク指向型ダイアログシステムは、インタラクティブなコミュニケーションを通じて特定のタスクを達成することを目指します。
   - DFA-RAGフレームワークは、従来の勾配ベースのトレーニングを必要としない独自のアプローチを導入し、シンプルさと適応性を提供します。
   - DFA-RAGは、現在のLLMアプリケーションの制約を解決し、カスタマーサービスなどの特定の領域で信頼性の高いコンテキストに適した応答を確保します。

3. **エージェント関連の提案**:
   - セマンティックルーターは、事前定義された意思決定レイヤーを備えたLLM技術の進展を表し、チャットボットやAIアシスタントを強化し

## 関連する文章自体の概要を出力する

In [15]:
pdf_dir = "./icml_2024_papers"

In [16]:
# 回答に使用したファイル
files

{'./icml_2024_embeddings/DFA-RAG: Conversational Semantic Router for Large Language Model with Definite Finite Automaton.csv',
 './icml_2024_embeddings/PinNet: Pinpoint Instructive Information for Retrieval Augmented Code-to-Text Generation.csv'}

In [17]:
# PDFファイルのパスを生成
pdf_paths = []
for file in files:
    file_name = os.path.basename(file)  # ファイル名を取得
    pdf_file_name = os.path.splitext(file_name)[0] + ".pdf"  # 拡張子を.csvから.pdfに変更
    pdf_file_path = os.path.join(pdf_dir, pdf_file_name)  # PDFファイルのパスを生成
    pdf_paths.append(pdf_file_path)

In [18]:
# 結果を表示
for pdf_path in pdf_paths:
    print(pdf_path)

./icml_2024_papers/PinNet: Pinpoint Instructive Information for Retrieval Augmented Code-to-Text Generation.pdf
./icml_2024_papers/DFA-RAG: Conversational Semantic Router for Large Language Model with Definite Finite Automaton.pdf


In [19]:
# PDFからテキストを抽出する関数
def convert_pdf_to_text(pdf_path):
    reader = PdfReader(pdf_path)
    text = ''
    for page in reader.pages:
        text += page.extract_text() + '\n'
    return text

In [20]:
# 各PDFファイルのテキストを取得して表示
pdf_texts = []
for pdf_path in pdf_paths:
    text = convert_pdf_to_text(pdf_path)
    pdf_texts.append({'file': pdf_path, 'text': text})


In [21]:
# pdf取得結果のテスト表示
for pdf in pdf_texts:
    print(f"File: {pdf['file']}")
    print(f"Text: {pdf['text'][:128]}...") 
    print("-"*64)

File: ./icml_2024_papers/PinNet: Pinpoint Instructive Information for Retrieval Augmented Code-to-Text Generation.pdf
Text: RankRAG: Unifying Context Ranking with
Retrieval-Augmented Generation in LLMs
Yue Yu∗
Georgia TechWei Ping∗
NVIDIAZihan Liu
NVID...
----------------------------------------------------------------
File: ./icml_2024_papers/DFA-RAG: Conversational Semantic Router for Large Language Model with Definite Finite Automaton.pdf
Text: DFA-RAG: Conversational Semantic Router for Large Language Model
with Definite Finite Automaton
Yiyou Sun1 2Junjie Hu1Wei Cheng2...
----------------------------------------------------------------


In [22]:
def generate_wiki_outline(model_name, scientific_paper):
    # Wikipediaページのアウトラインを生成する関数
    prompt = [
        {"role": "system", "content": "You are an experienced Wikipedia writer and want to edit a specific page."},
        {"role": "system", "content": "Your task is to write an outline of a Wikipedia page based on the content of a scientific article."},
        {"role": "system", "content": "Here is the format of your writing:"},
        {"role": "system", "content": "1. Use '#' Title ' to indicate section title, '##' Title ' to indicate subsection title, '###' Title ' to indicate subsubsection title, and so on."},
        {"role": "system", "content": "2. Do not include other information."},
        {"role": "user", "content": f"Scientific Paper: {scientific_paper}"},
        {"role": "system", "content": "Based on this information, please create a structured outline for the Wikipedia page."},
    ]

    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        temperature=TEMPERATURE,
    )
    
    outline = response.choices[0].message.content
    
    return outline


In [23]:
def generate_detailed_outline(model_name, outline, scientific_paper):
    """
    Given a summary and related search results, enhance a Wikipedia page outline with detailed descriptions.
    
    Parameters:
    model_name (str): The model to be used for generating the descriptions.
    summary_text (str): Summary text providing a concise overview of the topic.
    related_search_results (str): Text containing related search results or additional contextual information.
    outline (str): The basic outline of the Wikipedia page.
    
    Returns:
    str: The enhanced outline with detailed descriptions for each section.
    """
    prompt = [
        {"role": "system", "content": "You are tasked with enhancing a Wikipedia page outline by integrating detailed descriptions based on a scientific article."},
        {"role": "system", "content": "Here is the basic outline of the page you need to expand with detailed explanations:"},
        {"role": "user", "content": outline},
        {"role": "system", "content": "Use the following scientific article."},
        {"role": "user", "content": f"Scientific Paper: {scientific_paper}"},
        {"role": "system", "content": "Add a comprehensive description of each section based on the content of the scientific article, including an overview, an accurate description of the methodology, and relevant interpretations."}
    ]
    
    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        temperature=TEMPERATURE,
    )
    
    detailed_outline = response.choices[0].message.content
    
    return detailed_outline


In [24]:
%%time
# pdf取得結果のテスト表示
for pdf in pdf_texts:
    print(f"File: {pdf['file']}")
    outline_text = generate_wiki_outline(MODEL4o_NAME, pdf['text'])
    # 生成したアウトラインに詳細な説明を加える
    detailed_outline = generate_detailed_outline(MODEL4o_NAME, outline_text, outline_text)
    
    lang = detect(detailed_outline)
    if lang != "ja":
        translated_detailed_outline = translate_to_japanese(MODEL_NAME, detailed_outline)
    else:
        translated_detailed_outline = response
    print(translated_detailed_outline)
    print("-"*64)

File: ./icml_2024_papers/PinNet: Pinpoint Instructive Information for Retrieval Augmented Code-to-Text Generation.pdf
# RankRAG：LLMでのコンテキストランキングと検索拡張生成の統合

## 導入

### 概要
RankRAGフレームワークは、大規模言語モデル（LLMs）でコンテキストランキングと検索拡張生成（RAG）を統合することを目指しています。このアプローチは、コンテキストランキングメカニズムを統合することで、LLMsのコンテキストに即した回答を生成する能力を向上させます。この統合により、知識集約的なタスクにおいて特に、より正確でコンテクストに即した回答が得られるようになります。

### 検索拡張生成（RAG）の背景
検索拡張生成（RAG）は、検索メカニズムと生成モデルを組み合わせることで後者のパフォーマンスを向上させる手法です。検索コンポーネントは、外部コーパスから関連する文書やパッセージを選択し、生成モデルがこれを使用して回答を生成します。この方法は大規模データベースを活用し、最新かつ事実確実な情報を提供するため、生成モデルが高品質な出力を生成する能力が向上します。

### RankRAGの動機
RankRAGの動機は、既存のRAGモデルで観察される制限に起因します。これらのモデルは、コンテキストランキングでしばしば苦労することが多いです。多くのRAGモデルは、最も関連性の高い情報を効果的に優先付けできず、結果として最適でない回答生成となります。RankRAGは、この問題に対処するために、最も関連性の高いコンテキストが生成に使用されるようにする洗練されたランキングメカニズムを導入することで、回答の全体的な品質と正確性を向上させます。

## 開発と実装

### 著者と所属
RankRAGフレームワークは、人工知能と機械学習に特化した一流機関の研究者チームによって開発されました。所属機関には、自然言語処理（NLP）の分野への貢献で知られる名門大学や研究所が含まれています。

### 研究目標
RankRAG研究の主な目標は、効果的なコンテキストランキングメカニズムを検索拡張生成と統合することによって、LLMsのパフォーマンスを